In [ ]:
import duckdb
import numpy as np
import pandas as pd
import os

from pathlib import Path
from src.database.db_writer import Database
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize

def ingest_data():
    db_name = 'maine_legislation_and_testimony'
    db = Database(db_name)
    query = '''
    SELECT 
        Organization AS organization,
        COUNT(*) OVER (PARTITION BY Organization) AS count
    FROM TESTIMONY_HEADER th
    '''
    org_df = db.return_query_as_df(query)
    return org_df

def add_embeddings(idf, string_col):
    
    model_dir = str(Path(os.getcwd()).parents[1] / 'models' / 'all-minilm-l6')
    
    # Load sentence transformer model
    model = SentenceTransformer(model_dir)

    # Generate embeddings for unique organization names
    org_names = idf[string_col].unique()
    embeddings = np.array([model.encode(name) for name in org_names])
    normalized_embeddings = normalize(embeddings, norm='l2', axis=1)
    
    # Create a mapping of organization names to embeddings
    raw_embedding_map = dict(zip(org_names, embeddings))
    norm_embedding_map = dict(zip(org_names, normalized_embeddings))
    
    rdf = pd.DataFrame({
        'ORG_NAME': org_names, 
        'RAW_EMBEDDING': embeddings,
        'NORM_EMBEDDING': normalized_embeddings
    })
    
    return rdf

with duckdb.connect('../../data/maine_legislation_and_testimony.duckdb') as conn:
    
    df = ingest_data()
    # Example data (replace with your actual embeddings)
    texts = df.organization.unique()

    model_dir = str(Path(os.getcwd()).parents[1] / 'models' / 'all-minilm-l6')

    # Load sentence transformer model
    model = SentenceTransformer(model_dir)

    string_col = 'organization'
    # Generate embeddings for unique organization names
    org_names = df[string_col].unique()
    embeddings = np.array([model.encode(name) for name in org_names])
    normalized_embeddings = normalize(embeddings, norm='l2', axis=1)
    
    embeddings_df = pd.DataFrame(list(zip(org_names, embeddings, normalized_embeddings)), columns=['ORG_NAMES', 'RAW_EMBEDDINGS', 'NORM_EMBEDDINGS'])
    
    # Create the table schema if it doesn't exist
    conn.execute("CREATE TABLE IF NOT EXISTS ORG_EMBEDDINGS (ORG_NAME VARCHAR, RAW_EMBEDDING FLOAT[384], NORM_EMBEDDING FLOAT[384])")
    
    # Insert the embeddings into the DuckDB table
    conn.execute("INSERT INTO ORG_EMBEDDINGS SELECT * FROM embeddings_df")

print("Embeddings stored successfully in embeddings.db")

In [3]:
print(f'org: {len(org_names)} | raw: {len(embeddings)} | norm: {len(normalized_embeddings)}')

org: 15688 | raw: 15688 | norm: 15688


In [11]:
org_names[0:5]

array(['(City of) Rockland Public Library', '04765', '42 North Dental',
       "5210 Let's Go", '5G Free RI'], dtype=object)

In [16]:
list(zip(org_names, embeddings, normalized_embeddings))

ValueError: dictionary update sequence element #0 has length 3; 2 is required

In [18]:
pd.DataFrame(list(zip(org_names, embeddings, normalized_embeddings)), columns=['ORG_NAMES', 'RAW_EMBEDDINGS', 'NORM_EMBEDDINGS']).head()

,ORG_NAMES,RAW_EMBEDDINGS,NORM_EMBEDDINGS
0,(City of) Rockland Public Library,"[0.03466157, 0.036116432, -0.07471413, -0.021492843, 0.08646974, 0.025358453, -0.0825464, -0.066953644, -0.03503908, 0.08795211, 0.042841036, -0.009158235, -0.010215121, -0.074784175, -0.024299087, 0.024482993, 0.06035364, 0.079495214, 0.06161407, -0.036128942, -0.011742209, 0.07219678, 0.06404028, 0.009852927, -0.03995476, 0.04499916, -0.06830103, 0.006463658, -0.039008833, -0.024127223, 0.015468786, 0.0021644596, -0.013715002, -0.06427668, 0.08600131, 0.1169681, 0.018356789, -0.023875596, 0.040663213, 0.044161607, -0.08904536, -0.0033511075, 0.024336996, 0.010734489, -0.05257856, 0.00079668633, -0.008031198, -0.019091457, 0.025348129, 0.03258957, 0.04129991, 0.004128352, -0.03427839, -0.08407158, -0.021642694, 0.041471213, -0.06261719, 0.039903972, -0.04444611, -0.010102607, 0.02978635, -0.0034193601, -0.09714647, 0.048255704, 0.065770395, 0.0864933, -0.0029706561, 0.014010251, 0.032523677, -0.17419928, -0.042572137, 0.02178127, 0.020810446, -0.017883025, 0.03647422, 0.023327325, -0.037021574, 0.06142831, 0.030520953, -0.029681409, -0.006496469, -0.027707418, 0.09796527, -0.021136899, -0.074997954, 0.06339571, -0.0190656, 0.0038571428, -0.018850049, -0.00073602475, 0.08635044, -0.004428922, 0.038262762, 0.04304795, -0.046741083, -0.037743244, 0.017265901, 0.009138211, 0.015343162, 0.082085654, ...]","[0.03466157, 0.036116432, -0.07471413, -0.021492843, 0.08646974, 0.025358453, -0.0825464, -0.066953644, -0.03503908, 0.08795211, 0.042841036, -0.009158235, -0.010215121, -0.074784175, -0.024299087, 0.024482993, 0.06035364, 0.079495214, 0.06161407, -0.036128942, -0.011742209, 0.07219678, 0.06404028, 0.009852927, -0.03995476, 0.04499916, -0.06830103, 0.006463658, -0.039008833, -0.024127223, 0.015468786, 0.0021644596, -0.013715002, -0.06427668, 0.08600131, 0.1169681, 0.018356789, -0.023875596, 0.040663213, 0.044161607, -0.08904536, -0.0033511075, 0.024336996, 0.010734489, -0.05257856, 0.00079668633, -0.008031198, -0.019091457, 0.025348129, 0.03258957, 0.04129991, 0.004128352, -0.03427839, -0.08407158, -0.021642694, 0.041471213, -0.06261719, 0.039903972, -0.04444611, -0.010102607, 0.02978635, -0.0034193601, -0.09714647, 0.048255704, 0.065770395, 0.0864933, -0.0029706561, 0.014010251, 0.032523677, -0.17419928, -0.042572137, 0.02178127, 0.020810446, -0.017883025, 0.03647422, 0.023327325, -0.037021574, 0.06142831, 0.030520953, -0.029681409, -0.006496469, -0.027707418, 0.09796527, -0.021136899, -0.074997954, 0.06339571, -0.0190656, 0.0038571428, -0.018850049, -0.00073602475, 0.08635044, -0.004428922, 0.038262762, 0.04304795, -0.046741083, -0.037743244, 0.017265901, 0.009138211, 0.015343162, 0.082085654, ...]"
1,04765,"[-0.045759212, -0.04183617, -0.050981726, 0.056151208, -0.04686475, -0.013649284, 0.022174306, -0.0061217733, -0.01327612, -0.052265663, 0.010769392, -0.011942743, -0.03881265, -0.02613262, -0.047019683, 0.0118365595, -0.021841899, -0.09432463, -0.0065951385, -0.020528091, -0.08885713, 0.00018890749, -0.10452097, -0.002868008, -0.037003897, -0.061950404, 0.011506333, 0.0801614, -0.046418138, -0.07804905, 0.04966158, 0.09390082, 0.04968602, 0.00884832, 0.03804082, -0.013050593, 0.017889481, 0.0027592548, 0.03402378, 0.03035104, -0.04724185, -0.031397145, 0.0865735, 0.05300766, 0.006699013, 0.08647451, -0.046710204, 0.07193741, 0.05128108, 0.050209466, -0.014012984, 0.01441095, -0.056719378, 0.0536584, -0.017634096, -0.039088003, -0.03171945, -0.023167955, -0.022956373, 0.06553951, 0.10713625, -0.051308338, -0.0028523263, -0.03557816, 0.031828906, -0.015343453, -0.024439964, -0.16686326, 0.021443386, -0.06291768, -0.03216455, 0.08575748, 0.08126628, 0.0024995895, 0.05287017, -0.024491396, -0.0134059535, -0.0024990458, 0.041438755, -0.09239194, 0.01827174, -4.2462427e-05, -0.09782355, 0.0320102, -0.019972043, 0.03606366, 0.0022475433, 0.10830887, 0.039559312, 0.024998704, 0.0030508144, 0.09771043, 0.038255475, -0.04282571, -0.08877299, 0.08529399, 

In [19]:
rdf = pd.DataFrame(list(zip(org_names, embeddings, normalized_embeddings)), columns=['ORG_NAMES', 'RAW_EMBEDDINGS', 'NORM_EMBEDDINGS'])

rdf.head()

,ORG_NAMES,RAW_EMBEDDINGS,NORM_EMBEDDINGS
0,(City of) Rockland Public Library,"[0.03466157, 0.036116432, -0.07471413, -0.021492843, 0.08646974, 0.025358453, -0.0825464, -0.066953644, -0.03503908, 0.08795211, 0.042841036, -0.009158235, -0.010215121, -0.074784175, -0.024299087, 0.024482993, 0.06035364, 0.079495214, 0.06161407, -0.036128942, -0.011742209, 0.07219678, 0.06404028, 0.009852927, -0.03995476, 0.04499916, -0.06830103, 0.006463658, -0.039008833, -0.024127223, 0.015468786, 0.0021644596, -0.013715002, -0.06427668, 0.08600131, 0.1169681, 0.018356789, -0.023875596, 0.040663213, 0.044161607, -0.08904536, -0.0033511075, 0.024336996, 0.010734489, -0.05257856, 0.00079668633, -0.008031198, -0.019091457, 0.025348129, 0.03258957, 0.04129991, 0.004128352, -0.03427839, -0.08407158, -0.021642694, 0.041471213, -0.06261719, 0.039903972, -0.04444611, -0.010102607, 0.02978635, -0.0034193601, -0.09714647, 0.048255704, 0.065770395, 0.0864933, -0.0029706561, 0.014010251, 0.032523677, -0.17419928, -0.042572137, 0.02178127, 0.020810446, -0.017883025, 0.03647422, 0.023327325, -0.037021574, 0.06142831, 0.030520953, -0.029681409, -0.006496469, -0.027707418, 0.09796527, -0.021136899, -0.074997954, 0.06339571, -0.0190656, 0.0038571428, -0.018850049, -0.00073602475, 0.08635044, -0.004428922, 0.038262762, 0.04304795, -0.046741083, -0.037743244, 0.017265901, 0.009138211, 0.015343162, 0.082085654, ...]","[0.03466157, 0.036116432, -0.07471413, -0.021492843, 0.08646974, 0.025358453, -0.0825464, -0.066953644, -0.03503908, 0.08795211, 0.042841036, -0.009158235, -0.010215121, -0.074784175, -0.024299087, 0.024482993, 0.06035364, 0.079495214, 0.06161407, -0.036128942, -0.011742209, 0.07219678, 0.06404028, 0.009852927, -0.03995476, 0.04499916, -0.06830103, 0.006463658, -0.039008833, -0.024127223, 0.015468786, 0.0021644596, -0.013715002, -0.06427668, 0.08600131, 0.1169681, 0.018356789, -0.023875596, 0.040663213, 0.044161607, -0.08904536, -0.0033511075, 0.024336996, 0.010734489, -0.05257856, 0.00079668633, -0.008031198, -0.019091457, 0.025348129, 0.03258957, 0.04129991, 0.004128352, -0.03427839, -0.08407158, -0.021642694, 0.041471213, -0.06261719, 0.039903972, -0.04444611, -0.010102607, 0.02978635, -0.0034193601, -0.09714647, 0.048255704, 0.065770395, 0.0864933, -0.0029706561, 0.014010251, 0.032523677, -0.17419928, -0.042572137, 0.02178127, 0.020810446, -0.017883025, 0.03647422, 0.023327325, -0.037021574, 0.06142831, 0.030520953, -0.029681409, -0.006496469, -0.027707418, 0.09796527, -0.021136899, -0.074997954, 0.06339571, -0.0190656, 0.0038571428, -0.018850049, -0.00073602475, 0.08635044, -0.004428922, 0.038262762, 0.04304795, -0.046741083, -0.037743244, 0.017265901, 0.009138211, 0.015343162, 0.082085654, ...]"
1,04765,"[-0.045759212, -0.04183617, -0.050981726, 0.056151208, -0.04686475, -0.013649284, 0.022174306, -0.0061217733, -0.01327612, -0.052265663, 0.010769392, -0.011942743, -0.03881265, -0.02613262, -0.047019683, 0.0118365595, -0.021841899, -0.09432463, -0.0065951385, -0.020528091, -0.08885713, 0.00018890749, -0.10452097, -0.002868008, -0.037003897, -0.061950404, 0.011506333, 0.0801614, -0.046418138, -0.07804905, 0.04966158, 0.09390082, 0.04968602, 0.00884832, 0.03804082, -0.013050593, 0.017889481, 0.0027592548, 0.03402378, 0.03035104, -0.04724185, -0.031397145, 0.0865735, 0.05300766, 0.006699013, 0.08647451, -0.046710204, 0.07193741, 0.05128108, 0.050209466, -0.014012984, 0.01441095, -0.056719378, 0.0536584, -0.017634096, -0.039088003, -0.03171945, -0.023167955, -0.022956373, 0.06553951, 0.10713625, -0.051308338, -0.0028523263, -0.03557816, 0.031828906, -0.015343453, -0.024439964, -0.16686326, 0.021443386, -0.06291768, -0.03216455, 0.08575748, 0.08126628, 0.0024995895, 0.05287017, -0.024491396, -0.0134059535, -0.0024990458, 0.041438755, -0.09239194, 0.01827174, -4.2462427e-05, -0.09782355, 0.0320102, -0.019972043, 0.03606366, 0.0022475433, 0.10830887, 0.039559312, 0.024998704, 0.0030508144, 0.09771043, 0.038255475, -0.04282571, -0.08877299, 0.08529399, 

ValueError: All arrays must be of the same length